In [78]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import onnx
from onnx import helper
from onnx import AttributeProto, TensorProto, GraphProto

In [79]:
# Load model

model_path = 'C:/Users/t-agkum/onnxruntime/onnxruntime/python/tools/quantization/models/mobilenet_v1_1.0_224.onnx'
model = onnx.load(model_path)

print(len(model.graph.node))

120


In [80]:
# Add Rmin and Rmax nodes as outputs to all nodes 

added_nodes = []

for node in model.graph.node:
    if (node.op_type == "Conv") or (node.op_type == "MatMul"):
        input_name = node.name
        
        # rmin output node
        reduce_min_name = input_name + "_ReduceMin"
        reduce_min_node = onnx.helper.make_node("ReduceMin", [input_name + ":0"],
                        [reduce_min_name + ":0"], reduce_min_name, keepdims=0)
        added_nodes.append(reduce_min_node)

        # rmax output node
        reduce_max_name = input_name + "_ReduceMax"
        reduce_max_node = onnx.helper.make_node("ReduceMax", [input_name + ":0"],
                        [reduce_max_name + ":0"], reduce_max_name, keepdims=0)

        added_nodes.append(reduce_max_node)

model.graph.node.extend(added_nodes)
print(len(model.graph.node))

176


In [54]:
# Ensure intermediate nodes are added to graph output 

print(model.graph.output)

print(type(model.graph.output[0])) # type of each output

[name: "MobilenetV1/Predictions/Reshape_1:0"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_param: "unk__211"
      }
      dim {
        dim_value: 1001
      }
    }
  }
}
]
<class 'onnx.onnx_ml_pb2.ValueInfoProto'>


In [81]:
added_outputs = []

for node in model.graph.node:
    if (node.op_type == "ReduceMin") or (node.op_type == "ReduceMax"):
        #added_outputs.append(node.output)
        intermediate_output = helper.make_tensor_value_info(node.name + ":0", TensorProto.FLOAT, ())
        added_outputs.append(intermediate_output)

#print(added_outputs)
#print(type(added_outputs[0]))

model.graph.output.extend(added_outputs)
print(model.graph.output)

[name: "MobilenetV1/Predictions/Reshape_1:0"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_param: "unk__211"
      }
      dim {
        dim_value: 1001
      }
    }
  }
}
, name: "Conv__153_ReduceMin:0"
type {
  tensor_type {
    elem_type: 1
    shape {
    }
  }
}
, name: "Conv__153_ReduceMax:0"
type {
  tensor_type {
    elem_type: 1
    shape {
    }
  }
}
, name: "Conv__155_ReduceMin:0"
type {
  tensor_type {
    elem_type: 1
    shape {
    }
  }
}
, name: "Conv__155_ReduceMax:0"
type {
  tensor_type {
    elem_type: 1
    shape {
    }
  }
}
, name: "Conv__157_ReduceMin:0"
type {
  tensor_type {
    elem_type: 1
    shape {
    }
  }
}
, name: "Conv__157_ReduceMax:0"
type {
  tensor_type {
    elem_type: 1
    shape {
    }
  }
}
, name: "Conv__159_ReduceMin:0"
type {
  tensor_type {
    elem_type: 1
    shape {
    }
  }
}
, name: "Conv__159_ReduceMax:0"
type {
  tensor_type {
    elem_type: 1
    shape {
    }
  }
}
, name: "Conv__161_ReduceMin:

In [85]:
# Print all node names
node_names = []
for node in model.graph.node:
    if (node.op_type == "ReduceMin") or (node.op_type == "ReduceMax"):
        node_names.append(node.name)

print(node_names)
print(len(node_names))

['Conv__153_ReduceMin', 'Conv__153_ReduceMax', 'Conv__155_ReduceMin', 'Conv__155_ReduceMax', 'Conv__157_ReduceMin', 'Conv__157_ReduceMax', 'Conv__159_ReduceMin', 'Conv__159_ReduceMax', 'Conv__161_ReduceMin', 'Conv__161_ReduceMax', 'Conv__163_ReduceMin', 'Conv__163_ReduceMax', 'Conv__165_ReduceMin', 'Conv__165_ReduceMax', 'Conv__167_ReduceMin', 'Conv__167_ReduceMax', 'Conv__169_ReduceMin', 'Conv__169_ReduceMax', 'Conv__171_ReduceMin', 'Conv__171_ReduceMax', 'Conv__173_ReduceMin', 'Conv__173_ReduceMax', 'Conv__175_ReduceMin', 'Conv__175_ReduceMax', 'Conv__177_ReduceMin', 'Conv__177_ReduceMax', 'Conv__179_ReduceMin', 'Conv__179_ReduceMax', 'Conv__181_ReduceMin', 'Conv__181_ReduceMax', 'Conv__183_ReduceMin', 'Conv__183_ReduceMax', 'Conv__185_ReduceMin', 'Conv__185_ReduceMax', 'Conv__187_ReduceMin', 'Conv__187_ReduceMax', 'Conv__189_ReduceMin', 'Conv__189_ReduceMax', 'Conv__191_ReduceMin', 'Conv__191_ReduceMax', 'Conv__193_ReduceMin', 'Conv__193_ReduceMax', 'Conv__195_ReduceMin', 'Conv__195

In [39]:
# ORIGINAL

"""
# Add Rmin and Rmax nodes as outputs to all nodes 

added_nodes = []

for node in model.graph.node:
    input_name = node.name
    
    # rmin output node
    reduce_min_name = input_name + "_ReduceMin"
    reduce_min_node = onnx.helper.make_node("ReduceMin", [input_name + ":0"],
                    [reduce_min_name + ":0"], reduce_min_name, keepdims=0)
    added_nodes.append(reduce_min_node)
    
    # rmax output node
    reduce_max_name = input_name + "_ReduceMax"
    reduce_max_node = onnx.helper.make_node("ReduceMax", [input_name + ":0"],
                    [reduce_max_name + ":0"], reduce_max_name, keepdims=0)

    added_nodes.append(reduce_max_node)

model.graph.node.extend(added_nodes)
print(len(model.graph.node))
"""

360


In [82]:
# Save changed model for visualization in Netron
onnx.save(model, 'C:/Users/t-agkum/onnxruntime/onnxruntime/python/tools/quantization/models/mobilenet_v1_1.0_224_modified_full_output.onnx')


In [25]:
# Adding single rmin and rmax to first Conv (for testing)

added_nodes = []

# Rmin node
input_name = 'Conv__153' 
reduce_min_name = input_name + "_ReduceMin"
reduce_min_node = onnx.helper.make_node("ReduceMin", [input_name + ":0"],
                    [reduce_min_name + ":0"], reduce_min_name, keepdims=0)

added_nodes.append(reduce_min_node) 

# rmax node
reduce_max_name = input_name + "_ReduceMax"
reduce_max_node = onnx.helper.make_node("ReduceMax", [input_name + ":0"],
                    [reduce_max_name + ":0"], reduce_max_name, keepdims=0)

added_nodes.append(reduce_max_node) 

model.graph.node.extend(added_nodes)

print(len(model.graph.node))

122


In [26]:
# Save changed model for visualization in Netron
onnx.save(model, 'C:/Users/t-agkum/onnxruntime/onnxruntime/python/tools/quantization/models/mobilenet_v1_1.0_224_modified.onnx')



In [41]:
# Getting node names
for node in model.graph.node:
    #print(node.name)
    print(node.op_type)


Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Constant
Conv
Clip
Conv
Clip
Conv
Clip
Conv
Clip
Conv
Clip
Conv
Clip
Conv
Clip
Conv
Clip
Conv
Clip
Conv
Clip
Conv
Clip
Conv
Clip
Conv
Clip
Conv
Clip
Conv
Clip
Conv
Clip
Conv
Clip
Conv
Clip
Conv
Clip
Conv
Clip
Conv
Clip
Conv
Clip
Conv
Clip
Conv
Clip
Conv
Clip
Conv
Clip
Conv
Clip
AveragePool
Conv
Squeeze
Shape
Cast
Cast
Reshape
Softmax
Reshape
ReduceMin
ReduceMax
ReduceMin
ReduceMax
ReduceMin
ReduceMax
ReduceMin
ReduceMax
ReduceMin
ReduceMax
ReduceMin
ReduceMax
ReduceMin
ReduceMax
ReduceMin
Re